# Building Your First AI Crew with crewAI (The Modern Way)

## 1. Introduction & Lab Goals

Welcome to the next step in your AI agent journey! This notebook will guide you through using **crewAI**, a sophisticated framework designed to orchestrate multiple AI agents. We will be using the latest, most streamlined method for connecting to Large Language Models (LLMs).

Our goal is to build a simple **Research Crew** multi-agent workflow that researches a topic and writes a report.

The logic flow will look like this:

<img src="../img_1.png" alt="description" width="500">

## 2. Prerequisites

* **Knowledge:** Basic understanding of Python programming.
* **API Key and Datarobot Endpoint:** This lab is configured for **Datarobot LLM Gateway**. For external IDE use, you will need a datarobot API credentials, and Datarobot API endpoint. You can set these environment variables, as indicated below.
* **Python Version:** These labs are intended to be run on `python 3.11` or `python 3.12.` Others may work, but are untested.

### Step 1: Project Setup and Installation

First, let's install the necessary libraries. All the required libraries are included in ../requirements.txt
If you do not have uv installed, please unremark the first line.

In [1]:
#!pip install uv
!uv pip install -r ../../requirements.txt

Using Python 3.12.7 environment at: /Users/john.morrow/Documents/agentic-enablement/.venv
Audited 14 packages in 55ms


### First, we will connect to the Datarobot LLM Gateway and list the available LLMs.
The code below will connect to ANY of the LLMs hosted in the Datarobot LLM Gateway. For direct connections to external or customer hosted LLMS, you'll need to populate accordingly.

In [2]:
import datarobot as dr
import os
from dotenv import load_dotenv
from pprint import pprint
import random


load_dotenv()

import openai
#grab dr client credentials from os environment variables

dr_client = dr.Client(endpoint=os.getenv('DATAROBOT_ENDPOINT'), token=os.getenv('DATAROBOT_API_TOKEN'))

#set LLM Gateway
DR_API_TOKEN = dr_client.token
LLM_GATEWAY_BASE_URL = f"{dr_client.endpoint}/genai/llmgw"
response = dr_client.get(url="genai/llmgw/catalog/")
catalog = response.json()["data"]

response = dr_client.get(url="genai/llmgw/catalog/")
data = response.json()["data"]
supported_llms = [llm_model["model"] for llm_model in data]

print("These are list of LLMs supported by the Datarobot LLM Gateway:")
pprint(supported_llms)


These are list of LLMs supported by the Datarobot LLM Gateway:
['azure/gpt-4o-2024-11-20',
 'azure/gpt-4o-mini',
 'azure/gpt-4-turbo',
 'azure/gpt-4-32k',
 'azure/gpt-4',
 'azure/gpt-35-turbo',
 'azure/o3-mini',
 'azure/o1-mini',
 'azure/o1',
 'bedrock/amazon.nova-premier-v1:0',
 'bedrock/amazon.nova-pro-v1:0',
 'bedrock/amazon.nova-lite-v1:0',
 'bedrock/amazon.nova-micro-v1:0',
 'bedrock/amazon.titan-text-express-v1',
 'bedrock/anthropic.claude-opus-4-1-20250805-v1:0',
 'bedrock/anthropic.claude-opus-4-20250514-v1:0',
 'bedrock/anthropic.claude-sonnet-4-20250514-v1:0',
 'bedrock/anthropic.claude-3-7-sonnet-20250219-v1:0',
 'bedrock/anthropic.claude-3-5-sonnet-20241022-v2:0',
 'bedrock/anthropic.claude-3-5-sonnet-20240620-v1:0',
 'bedrock/anthropic.claude-3-5-haiku-20241022-v1:0',
 'bedrock/anthropic.claude-3-opus-20240229-v1:0',
 'bedrock/anthropic.claude-3-sonnet-20240229-v1:0',
 'bedrock/anthropic.claude-3-haiku-20240307-v1:0',
 'bedrock/anthropic.claude-v2:1',
 'bedrock/cohere.comm

### Next, we are going to choose a random entry from the list of LLMS that was returned:

In [2]:
client = openai.OpenAI(
    api_key=dr_client.token,
    base_url=dr_client.endpoint + "/genai/llmgw"
)

# We are going to select a RANDOM model from the list above... feel free to hard code your preference!!!
#model = random.choice(supported_llms)
model = 'vertex_ai/gemini-2.0-flash-001'
#model = 'bedrock/meta.llama3-70b-instruct-v1:0'
response = client.chat.completions.create(
    model=model,
     messages= [
    {
      "role": "user",
      "content": f"Who are you? {model}"
    }
  ]
)
pprint(model)
pprint(response.choices[0])

'vertex_ai/gemini-2.0-flash-001'
Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I am a large language model, trained by Google.\n', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))


In [3]:
#use this for troubleshooting endpoint and token issues
print(dr_client.token)
print(dr_client.endpoint + "/genai/llmgw")

NjdmNTM2NjRiOWM5NjhjMzYzYzkzNTA4Omt2UXd6MkZnYWZqWDBia1lCSE5kMmh0b2drSW1YeGdhMlZ6L2pFTUdiVFk9
https://app.datarobot.com/api/v2/genai/llmgw


### Step 2: Create a Custom DuckDuckGo Search Tool

Since the CrewAI DuckDuckGoSearchRunTool is not available in the current version, we'll create our own custom search tool using the `duckduckgo-search` library directly.

In [4]:
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type
from ddgs import DDGS
from datetime import datetime
import json

class SearchInput(BaseModel):
    """Input schema for DuckDuckGo search tool."""
    query: str = Field(..., description="The search query")

class DuckDuckGoSearchTool(BaseTool):
    name: str = "duckduckgo_search"
    description: str = "Search the web using DuckDuckGo for current information"
    args_schema: Type[BaseModel] = SearchInput

    def _run(self, query: str) -> str:
        """Execute the search and return formatted results."""
        try:
            with DDGS() as ddgs:
                # Try news search first
                news_results = list(ddgs.news(query, max_results=5))

                if news_results:
                    formatted_results = []
                    formatted_results.append(f"Search Results for: {query}")
                    formatted_results.append(f"Search performed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                    formatted_results.append("\n=== NEWS RESULTS ===")

                    for i, result in enumerate(news_results, 1):
                        title = result.get('title', 'No title')
                        body = result.get('body', '')
                        url = result.get('url', '')
                        date = result.get('date', 'Recent')

                        formatted_results.append(f"\n{i}. {title}")
                        if body:
                            # Truncate body to keep it manageable
                            body_snippet = body[:300] + "..." if len(body) > 300 else body
                            formatted_results.append(f"   Summary: {body_snippet}")
                        formatted_results.append(f"   Date: {date}")
                        formatted_results.append(f"   Source: {url}")

                    return "\n".join(formatted_results)

                # If no news results, try regular web search
                web_results = list(ddgs.text(query, max_results=5))

                if web_results:
                    formatted_results = []
                    formatted_results.append(f"Search Results for: {query}")
                    formatted_results.append(f"Search performed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                    formatted_results.append("\n=== WEB RESULTS ===")

                    for i, result in enumerate(web_results, 1):
                        title = result.get('title', 'No title')
                        body = result.get('body', '')
                        url = result.get('href', '')

                        formatted_results.append(f"\n{i}. {title}")
                        if body:
                            # Truncate body to keep it manageable
                            body_snippet = body[:300] + "..." if len(body) > 300 else body
                            formatted_results.append(f"   Summary: {body_snippet}")
                        formatted_results.append(f"   Source: {url}")

                    return "\n".join(formatted_results)

                else:
                    return f"No search results found for: {query}"

        except Exception as e:
            return f"Search error for '{query}': {str(e)}"

# Create the search tool instance
search_tool = DuckDuckGoSearchTool()
print("✅ Custom DuckDuckGo search tool created successfully!")

✅ Custom DuckDuckGo search tool created successfully!


### Step 3: Define the Crew

Now we define the agents and tasks. This is the heart of crewAI - creating specialized agents that can work together to accomplish complex goals.

**📚 For sample configuration, refer to the [CrewAI Research Lab Setup Guide](../CrewAI_ResearchLab_Setup.md)**

#### Key Components You'll Create:

1. **LLM Configuration** - Set up your connection to the DataRobot LLM Gateway
2. **Agents** - Define specialized AI workers with distinct roles and capabilities
3. **Tasks** - Create specific assignments for each agent to complete
4. **Crew** - Organize agents and tasks into a collaborative workflow

#### Understanding the Architecture:

- **Agent**: A specialized AI worker with a specific role (e.g., Researcher, Writer)
- **Task**: A specific job assigned to an agent with clear expectations
- **Tool**: Capabilities you give to agents (like our search tool)
- **Crew**: The orchestrator that manages agents and tasks in sequence

#### Important Configuration Notes:

- `verbose=True/False`: Controls output verbosity (set to False for DataRobot compatibility)
- `allow_delegation=True/False`: Whether agents can delegate tasks to other agents
- `custom_llm_provider="openai"`: Required for DataRobot LLM Gateway compatibility
- `process=Process.sequential`: Tasks execute one after another (vs. parallel)

**Complete and Run the code below to create your research crew:**

In [5]:
import os
from crewai import Agent, Task, Crew, Process, LLM

# --- LLM Configuration for DataRobot Gateway ---


llm = LLM(
    model=model,
    #model='vertex_ai/claude-3-7-sonnet@20250219',
    api_key=dr_client.token,
    base_url=dr_client.endpoint + "/genai/llmgw",
    custom_llm_provider="openai"  # Force liteLLM to treat this as OpenAI
)

# --- Agent & Task Definitions ---
# Define the Researcher Agent
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory='''You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting
  actionable insights.''',
  verbose=True,  # Set to False for DataRobot compatibility
  allow_delegation=False,
  tools=[search_tool],  # Using our custom search tool
  llm=llm  # Pass the LLM object, not a string
)

# Define the Writer Agent
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory='''You are a renowned Content Strategist, known for
  your insightful and engaging articles.
  You transform complex concepts into compelling narratives.''',
  verbose=True,  # Set to False for DataRobot compatibility
  allow_delegation=True,
  llm=llm  # Pass the LLM object, not a string
)

# Create the Research Task
research_task = Task(
  description='''Conduct a comprehensive analysis of the latest advancements in AI for 2025.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Use the search tool to find recent information about AI developments.
  Your final answer MUST be a full analysis report.''',
  expected_output='A comprehensive 3-paragraph report on the latest AI advancements.',
  agent=researcher
)

# Create the Writing Task
writing_task = Task(
  description='''Using the research analyst\'s report, compose an engaging blog post.
  The post should be easy to understand, insightful, and positive in tone.
  Make it sound cool, avoid complex words so it doesn't sound like AI.
  Your final answer MUST be the full blog post of at least 4 paragraphs.''',
  expected_output='A 4-paragraph blog post on AI advancements, formatted in markdown.',
  agent=writer
)

# --- Crew Definition ---
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, writing_task],
  process=Process.sequential,
  verbose=False  # Set to False for DataRobot compatibility
)

print("✅ Crew and tasks are defined and ready to run.")

✅ Crew and tasks are defined and ready to run.


### Step 4: Run Your AI Crew

With everything set up, let's kick off the crew's work. This will start the sequential process, and you will see the agents' thoughts and actions in the output below.

In [21]:
result = crew.kickoff()

print("\n\n########################")
print("## Here is your crew's result:")
print("########################\n")
print(result)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Conduct a comprehensive analysis of the latest advancements in AI for 2025.                              │
│    Identify key trends, breakthrough technologies, and potential industry impacts.                              │
│    Use the search tool to find recent information about AI developments.                                        │
│    Your final answer MUST be a full analysis report.                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Thought: Okay, I need to conduct a comprehensive analysis of the latest AI advancements expected for 2025.     │
│  I'll use the search tool to gather information on key trends, breakthrough technologies, and potential         │
│  industry impacts. I'll focus on recent developments and predictions for 2025 to create a detailed report.      │
│                                                                                                                 │
│  Using Tool: duckduckgo_search                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"AI advancements 2025\"}"                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Search Results for: AI advancements 2025                                                                       │
│  Search performed at: 2025-07-17 12:46:53                                                                       │
│                                                                                                                 │
│  === NEWS RESULTS ===                                                                                           │
│                                                                                                                 │
│  1. AI advancements must not leave developing nations behind                                                    │
│     Summary: Artificial Intelligence (AI) and rapid technological advancements are changing the global          │
│  economic landscape, but policymakers must ensure that this shift does not deepen inequality or leave           │
│  developing nations behind.                                                                                     │
│     Date: 2025-07-17T13:18:00+00:00                                                                             │
│     Source: https://www.sanews.gov.za/south-africa/ai-advancements-must-not-leave-developing-nations-behind     │
│                                                                                                                 │
│  2. Target Drones Outlook Report 2025 | Precision Training Revolution - AI and Swarm Technologies to Propel     │
│  the Market to $13.9 Billion by 2034                                                                            │
│     Summary: The Target Drones Market, valued at USD 6.8B in 2025, is set to expand to USD 13.9B by 2034,       │
│  growing at a CAGR of 8.4%. As a critical asset in military training, these drones simulate aerial threats,     │
│     Date: 2025-07-17T13:23:00+00:00                                                                             │
│     Source: https://finance.yahoo.com/news/target-drones-outlook-report-2025-132300296.html                     │
│                                                                                                                 │
│  3. The AI Revolution In Healthcare: How Data Science Is Transforming Drug Discovery And Medical Diagnosis      │
│     Summary: Traditionally, drug discovery relied heavily on trial and error, with long timelines and high      │
│  costs. The introduction of biostatistics helped make this process more rigorous, but the pace remained slow.   │
│  Today, AI and data science are streamlining every phase, from early molecular analysis to clinica...           │
│     Date: 2025-07-17T11:30:00+00:00                                                                             │
│     Source:                                                                                                     │
│  https://www.forbes.com/councils/forbestechcouncil/2025/07/17/the-ai-revolution-in-healthcare-how-data-science  │
│  -is-transforming-drug-discovery-and-medical-diagnosis/                                                         │
│                                                                                                                 │
│  4. The 10 Hottest SaaS Startup Companies Of 2025 (So Far)                                                      │
│     Summary: Anthropic, MaintainX and Persona are just some of the startups to make CRN's list of the 10        │
│  hottest Software-as-a-Service startup companies of 2025 so far, selected for offering users cutting-edge       │
│  technology under a familiar subscription-based, updated-online SaaS model.                                     │
│    ...                                                

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The year 2025 is witnessing significant advancements across various sectors, driven by breakthroughs in        │
│  Artificial Intelligence. AI is increasingly integrated into manufacturing processes, enhancing efficiency and  │
│  power through optimized equipment maintenance, quality control, production scheduling, and supply chain        │
│  management. In healthcare, AI and data science are revolutionizing drug discovery and medical diagnosis,       │
│  streamlining every phase from molecular analysis to clinical trials. Furthermore, the SaaS landscape is        │
│  experiencing a surge in innovative companies leveraging AI to deliver cutting-edge technology through          │
│  subscription-based models.                                                                                     │
│                                                                                                                 │
│  Beyond specific industries, AI advancements are also influencing broader infrastructural and economic          │
│  landscapes. There is a growing emphasis on ensuring that AI's benefits extend to developing nations,           │
│  preventing increased inequality. The development of AI-ready data centers, such as the recent acquisition by   │
│  DataBank in Atlanta, underscores the growing demand for robust infrastructure to support AI applications.      │
│  Additionally, AI is transforming military training, with target drone markets experiencing substantial growth  │
│  due to the integration of AI and swarm technologies.                                                           │
│                                                                                                                 │
│  Overall, the trajectory of AI in 2025 points towards a future where AI is not just a technological tool, but   │
│  a fundamental force reshaping industries, economies, and societies. While the advancements bring immense       │
│  potential, they also highlight the importance of ethical considerations, equitable distribution of benefits,   │
│  and the need for robust infrastructure to support these rapidly evolving technologies.                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tech Content Strategist                                                                                 │
│                                                                                                                 │
│  Thought: Thought: I need to create the blog post so that the Senior Research Analyst can review it.            │
│                                                                                                                 │
│  Using Tool: Delegate work to coworker                                                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"coworker\": \"Senior Research Analyst\", \"task\": \"I've crafted a blog post draft about the AI advancem  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  ```                                                                                                            │
│  ## AI is Taking Over (in a Good Way!) - What's Coming in 2025                                                  │
│                                                                                                                 │
│  Get ready, because 2025 is shaping up to be a wild year for Artificial Intelligence! AI is leaping into        │
│  pretty much every corner of our lives, making things faster, smarter, and more efficient. Let's break down     │
│  what's happening.                                                                                              │
│                                                                                                                 │
│  First up, factories are getting a major upgrade. AI is now in charge of keeping equipment running smoothly,    │
│  making sure products are top-notch, and even planning out the best way to get things done. Think of it like    │
│  this: AI is the super-smart manager that never sleeps, always looking for ways to improve how things are       │
│  made. For example, AI can predict when a machine might break down and schedule maintenance before it causes    │
│  problems, saving time and money.                                                                               │
│                                                                                                                 │
│  But that's not all. AI is also changing the game in healthcare. It's helping doctors discover new drugs and    │
│  diagnose diseases faster than ever before. From analyzing complex medical images to speeding up clinical       │
│  trials, AI is revolutionizing the whole process. Imagine AI helping to personalize cancer treatments based on  │
│  a patient's unique genetic makeup! And if you're into the latest tech, you'll love what's happening with SaaS  │
│  companies. They're using AI to create some seriously cool subscription-based services.                         │
│                                                                                                                 │
│  Of course, with all this awesome AI stuff happening, it's super important to talk about the ethics. We need    │
│  to make sure that everyone benefits from AI, not just a select few. This means thinking about things like      │
│  fairness, privacy, and making sure AI doesn't accidentally discriminate against anyone. For instance, if AI    │
│  is used to decide who gets a loan, we need to ensure it's not biased against certain groups of people. Or,     │
│  when AI is used in hiring, that it's evaluating candidates based on skills and qualifications, not on gender   │
│  or ethnicity. Data privacy is also paramount; we need strong safeguards to protect personal information used   │
│  by AI systems. It's a big responsibility, but if we do it right, AI c...                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tech Content Strategist                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  ## AI is Taking Over (in a Good Way!) - What's Coming in 2025                                                  │
│                                                                                                                 │
│  Get ready, because 2025 is shaping up to be a wild year for Artificial Intelligence! AI is leaping into        │
│  pretty much every corner of our lives, making things faster, smarter, and more efficient. Let's break down     │
│  what's happening.                                                                                              │
│                                                                                                                 │
│  First up, factories are getting a major upgrade. AI is now in charge of keeping equipment running smoothly,    │
│  making sure products are top-notch, and even planning out the best way to get things done. Think of it like    │
│  this: AI is the super-smart manager that never sleeps, always looking for ways to improve how things are       │
│  made. For example, AI can predict when a machine might break down and schedule maintenance before it causes    │
│  problems, saving time and money.                                                                               │
│                                                                                                                 │
│  But that's not all. AI is also changing the game in healthcare. It's helping doctors discover new drugs and    │
│  diagnose diseases faster than ever before. From analyzing complex medical images to speeding up clinical       │
│  trials, AI is revolutionizing the whole process. Imagine AI helping to personalize cancer treatments based on  │
│  a patient's unique genetic makeup! And if you're into the latest tech, you'll love what's happening with SaaS  │
│  companies. They're using AI to create some seriously cool subscription-based services.                         │
│                                                                                                                 │
│  Of course, with all this awesome AI stuff happening, it's super important to talk about the ethics. We need    │
│  to make sure that everyone benefits from AI, not just a select few. This means thinking about things like      │
│  fairness, privacy, and making sure AI doesn't accidentally discriminate against anyone. For instance, if AI    │
│  is used to decide who gets a loan, we need to ensure it's not biased against certain groups of people. Or,     │
│  when AI is used in hiring, that it's evaluating candidates based on skills and qualifications, not on gender   │
│  or ethnicity. Data privacy is also paramount; we need strong safeguards to protect personal information used   │
│  by AI systems. It's a big responsibility, but if we do it right, AI can make the world a better place for      │
│  everyone. We should carefully consider AI's impact on trust, fairness, accountability and safety. As AI        │
│  becomes more prevalent, transparency in how these systems work is key to building and maintaining trust.       │
│                                                                                                                 │
│  So, as AI continues to explode, get ready for a smarte



########################
## Here is your crew's result:
########################

```markdown
## AI is Taking Over (in a Good Way!) - What's Coming in 2025

Get ready, because 2025 is shaping up to be a wild year for Artificial Intelligence! AI is leaping into pretty much every corner of our lives, making things faster, smarter, and more efficient. Let's break down what's happening.

First up, factories are getting a major upgrade. AI is now in charge of keeping equipment running smoothly, making sure products are top-notch, and even planning out the best way to get things done. Think of it like this: AI is the super-smart manager that never sleeps, always looking for ways to improve how things are made. For example, AI can predict when a machine might break down and schedule maintenance before it causes problems, saving time and money.

But that's not all. AI is also changing the game in healthcare. It's helping doctors discover new drugs and diagnose diseases faster than ever befor

## 6. Testing the Search Tool

Let's test our custom search tool to make sure it works correctly:

In [ ]:
# Test the search tool directly
test_query = "latest AI developments coming in 2026"
test_result = search_tool._run(test_query)

print("=== SEARCH TOOL TEST ===")
print(f"Query: {test_query}")
print("\nResults:")
print(test_result[:5000] + "..." if len(test_result) > 5000 else test_result)
print("\n=== TEST COMPLETED ===")